## Run Inference

First, compile the model. Don't worry ahout the warning about disabiling the optimized runtime, this is an auxilary engine used to handle any odd input shapes we get.

In [15]:
from deepsparse.sentence_transformers import DeepSparseSentenceTransformer
ds_model = DeepSparseSentenceTransformer("zeroshot/bge-base-en-v1.5-quant")

2023-11-01 14:22:26 deepsparse.engine WARNING  batch_size < 1 so disabling batch size override
[nm_ort 7f2edf331740 >WARN<  is_supported_graph src/onnxruntime_neuralmagic/supported/ops.cc:150] Warning: Optimized runtime disabled - Detected dynamic input input_ids dim 0. Set inputs to static shapes to enable optimal performance.


Run inference by calling encode.

In [30]:
# Our sentences we like to encode
sentences = [
    'This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.'
]

# Sentences are encoded by calling model.encode()
embeddings = ds_model.encode(
    sentences,
    normalize_embeddings=True
)

# Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding Shape:", embedding.shape)
    print("")

1


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Sentence: This framework generates embeddings for each input sentence
Embedding Shape: (768,)

Sentence: Sentences are passed as a list of string.
Embedding Shape: (768,)

Sentence: The quick brown fox jumps over the lazy dog.
Embedding Shape: (768,)



In [31]:
import numpy as np
np.linalg.norm(embedding)

1.0

## Benchmark Performance

DeepSparse offers superior performance relative to the baseline of sentence-transformers.

In [7]:
import time

def benchmark_model(model, sentences):
    # Benchmark the encoding time for a model with a given list of sentences.
    start_time = time.time()
    _ = model.encode(sentences)
    elapsed_time = time.time() - start_time
    return elapsed_time

In [6]:
import sentence_transformers
st_model = sentence_transformers.SentenceTransformer("BAAI/bge-base-en-v1.5")

In [18]:
from deepsparse.sentence_transformers.benchmark_encoding import generate_random_sentence

num_sentences = 100
sentence_length = 700
sentences = [generate_random_sentence(sentence_length) for _ in range(num_sentences)]

In [9]:
st_time = benchmark_model(st_model, sentences)
print(f"sentence_transformer time: {st_time}")

sentence_transformer time: 25.30896258354187


In [19]:
ds_time = benchmark_model(ds_model, sentences)
print(f"sentence_transformer time: {ds_time}")

1


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

sentence_transformer time: 6.328191041946411


## Run Evaluation

We provided a script showing how to run an MTEB evaluation.

In [1]:
!mteb --available_tasks

INFO:mteb.cmd:Running with parameters: Namespace(model=None, task_types=None, task_categories=None, tasks=None, task_langs=None, device=None, batch_size=32, seed=42, output_folder='results', verbosity=2, eval_splits=None, k=None, n_experiments=None, samples_per_label=None, corpus_chunk_size=None, available_tasks=True)
───────────────────────────────── MTEB tasks  ──────────────────────────────────
Classification
    - AmazonCounterfactualClassification, s2s, multilingual 4 / 4 langs
    - AmazonPolarityClassification, p2p
    - AmazonReviewsClassification, s2s, multilingual 6 / 6 langs
    - AngryTweetsClassification, s2s
    - Banking77Classification, s2s
    - DalajClassification, s2s
    - DanishPoliticalCommentsClassification, s2s
    - DKHateClassification, s2s
    - EmotionClassification, s2s
    - ImdbClassification, p2p
    - LccSentimentClassification, s2s
    - MassiveIntentClassification, s2s, multilingual 51 / 51 langs
    - MassiveScenarioClassification, s2s, multilingual 

In [5]:
!python3 run_eval.py \
    --model-name zeroshot/bge-base-en-v1.5-quant \
    --task-names STS12 STS13 \
    --output-dir results/bge-base-quant

2023-11-01 14:08:17 deepsparse.engine WARNING  batch_size < 1 so disabling batch size override
DeepSparse, Copyright 2021-present / Neuralmagic, Inc. version: 1.6.0.20231027 COMMUNITY | (2083fc4e) (release) (optimized) (system=avx512_vnni, binary=avx512)
[nm_ort 7fcb1789a740 >WARN<  is_supported_graph src/onnxruntime_neuralmagic/supported/ops.cc:150] Warning: Optimized runtime disabled - Detected dynamic input input_ids dim 0. Set inputs to static shapes to enable optimal performance.
─────────────────────────────── Selected tasks  ────────────────────────────────
STS
    - STS12, s2s
    - STS13, s2s


1
Batches: 100%|██████████████████████████████| 3108/3108 [01:28<00:00, 35.11it/s]
1
Batches: 100%|██████████████████████████████| 3108/3108 [01:11<00:00, 43.57it/s]
1
Batches: 100%|██████████████████████████████| 1500/1500 [00:30<00:00, 49.33it/s]
1
Batches: 100%|██████████████████████████████| 1500/1500 [00:41<00:00, 35.85it/s]
{'STS12': {'mteb_version': '1.1.1', 'dataset_revision': '